<a href="https://colab.research.google.com/github/sasferi95/Kmeans/blob/master/YOLOv3_vehicle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get YOLO and test if its works

In [0]:
!git clone https://github.com/AlexeyAB/darknet.git

In [0]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [0]:
!/usr/local/cuda/bin/nvcc --version

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [0]:
!make

In [0]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

In [0]:
import cv2
from google.colab.patches import cv2_imshow
image = cv2.imread("predictions.jpg")
img2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2_imshow(img2)

# Get training data

In [0]:
!wget http://podoce.dinf.usherbrooke.ca/static/dataset/MIO-TCD-Localization.tar

In [0]:

# 4/wwEOGmqdqkJBtqbnrgHOtuyGsGejaFnJhKiIdQp796QKq_DKEDH5Smc
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
path = "/content/drive/My Drive/Resources/Train"
!cd path 

In [0]:
!cd drive/My\ Drive/Resources/Train


In [0]:
!tar -xf MIO-TCD-Localization.tar -C /content/drive/My\ Drive/Resources/Train

Create directory and unpack the files there

In [0]:
!mkdir /content/resources

In [0]:
!tar -xf MIO-TCD-Localization.tar -C /content/resources

# Prepare training data

In [1]:
import cv2
import os
import csv
import shutil
from collections import defaultdict

typedictionary={
"articulated_truck": 0,
"bus" : 1,
"car":2,
"motorcycle":3,
"motorized_vehicle":4,
"non-motorized_vehicle":5,
"pickup_truck":6,
"single_unit_truck":7,
"work_van":8
}

def create_dictionary_from_csv(filepath):
    with open(filepath) as file:
        reader = csv.reader(file, skipinitialspace=True)
        d = defaultdict(list)
        for row in reader:
            #to the appropiate "image"
            d[row[0]].append([row[1], int(row[2]), int(row[3]), int(row[4]), int(row[5])])
        return d

def Calculate_Yolo_Parameters(bboxx1,bboxy1,bboxx2,bboxy2, imagewidth, imageHeight):
    # center coordinates of bounding bounding box
    x = (bboxx1 + bboxx2)/2.0
    y = (bboxy1 + bboxy2)/2.0
    # "normalize" bounding box center coordinates
    centerx = x/imagewidth
    centery = y/imageHeight

    # normalized bounding box width/heigth
    # I might have to devide by 2 not sure
    bboxw = (bboxx2 - bboxx1)/imageHeight
    bboxh = (bboxy2 - bboxy1)/imagewidth
    return centerx, centery, bboxw, bboxh

def GetClassNumber(vehicletype):
    if 'vehicletype' in typedictionary.keys():
      return typedictionary[vehicletype]
    else:
      return None


def moveValidationData(fromfolder, destfolder, startingimgnum):
    if not os.path.exists(destfolder):
      os.makedirs(destfolder)
    for file in os.listdir(fromfolder):
      filename, extension = os.path.splitext(file)
      if int(filename) >= startingimgnum:
        shutil.move(os.path.join(fromfolder,file), os.path.join(destfolder, filename + extension))
        if int(filename) % 500 == 0:
          print(filename +"has been moved to the validation folder")

def generateTrainingData(groundtruthcsvpath, imagefolder, traingdata):
    gtdictionary = create_dictionary_from_csv(groundtruthcsvpath)#filepath
    for file in os.listdir(imagefolder):
        filename, extension = os.path.splitext(file)
        if extension in acceptedFormats:
            ffp = os.path.join(imagefolder, file)
            img = cv2.imread(ffp)
            imheight, imwidth, channels = img.shape
            bboxdataforimage = gtdictionary[filename]
            newfile = os.path.join(imagefolder,filename+".txt")
            if os.path.exists(newfile):
                os.remove(newfile)
            with open(newfile, "a") as file:
                for bbox in bboxdataforimage:
                        cx,cy,bboxw,bboxh = Calculate_Yolo_Parameters(bbox[1],bbox[2],bbox[3],bbox[4], imwidth, imheight)
                        vehicle = GetClassNumber(bbox[0])
                        if vehicle != None:
                          file.write("{} {} {} {} {}\n".format(vehicle,cx,cy,bboxw,bboxh))
                with open(traingdata, "a") as file:
                  file.write("{}\n".format(newfile+".jpg"))
            with open(newfile, "a") as file:
                  for bbox in bboxdataforimage:
                        cx,cy,bboxw,bboxh = Calculate_Yolo_Parameters(bbox[1],bbox[2],bbox[3],bbox[4], imwidth, imheight)
                        vehicle = GetClassNumber(bbox[0])
                        if vehicle != None:
                          file.write("{} {} {} {} {}\n".format(vehicle,cx,cy,bboxw,bboxh))
            if int(filename)%5000==0:
              print(filename)


traingdata = "/content/darknet/train.txt"
acceptedFormats=[".jpg"]
imagefolderpath = r"/content/resources/MIO-TCD-Localization/train"
groundtruthcsvpath = r"/content/resources/MIO-TCD-Localization/gt_train.csv"
validationfolder="/content/resources/MIO-TCD-Localization/myvalidation"


moveValidationData(imagefolderpath, validationfolder, 100527)
generateTrainingData(groundtruthcsvpath, imagefolderpath, traingdata)
generateTrainingData(groundtruthcsvpath, validationfolder, traingdata)


00090000
00030000
00070000
00045000
00060000
00005000
00040000
00020000
00025000
00035000
00075000
00015000
00065000
00000000
00050000
00095000
00055000
00100000
00085000
00080000
00010000
00105000
00110000


In [0]:
!printf printf 'articulated_truck\nbus\ncar\nmotorcycle\nmotorized_vehicle\nnon-motorized_vehicle\npickup_truck\nsingle_unit_truck\nwork_van' >> vehicle.names

In [2]:
!zip -r /content/myvalidation.zip /content/resources/MIO-TCD-Localization/myvalidation

Streaming output truncated to the last 5000 lines.
  adding: content/resources/MIO-TCD-Localization/myvalidation/00108925.txt (stored 0%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00109426.jpg (deflated 1%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00105085.txt (stored 0%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00110271.txt (stored 0%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00109699.jpg (deflated 4%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00104271.jpg (deflated 3%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00102240.jpg (deflated 2%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00108100.jpg (deflated 1%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00107259.txt (stored 0%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/00104290.txt (stored 0%)
  adding: content/resources/MIO-TCD-Localization/myvalidation/0010946

In [6]:
from google.colab import files
files.download("/content/resources/MIO-TCD-Localization/myvalidation")

MessageError: ignored